![title](keras.png)

![title](Chollet.png)

# What is Keras?

Keras is a high-level neural networks API, written in Python and capable of running on top of __TensorFlow__, __CNTK__, or __Theano__. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.

Use Keras if you need a deep learning library that:

+ Allows for easy and fast prototyping (through user friendliness, modularity, and extensibility).
+ Supports both convolutional networks and recurrent networks, as well as combinations of the two.
+ Runs seamlessly on CPU and GPU.

Read the documentation at Keras.io.

Keras is compatible with: Python 2.7-3.6.

# Multi-backend Keras and tf.keras

At this time, Keras users who use multi-backend Keras with the TensorFlow backend switch to __tf.keras__ in __TensorFlow 2.0__. tf.keras is better maintained and has better integration with TensorFlow features (eager execution, distribution support and other).

Keras 2.2.5 was the last release of Keras implementing the 2.2.* API. It was the last release to only support TensorFlow 1 (as well as Theano and CNTK).

The current release is Keras 2.3.0, which makes significant API changes and add support for TensorFlow 2.0. The 2.3.0 release will be the last major release of multi-backend Keras. Multi-backend Keras is superseded by tf.keras.

Bugs present in multi-backend Keras will only be fixed until April 2020 (as part of minor releases).

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

df_train = pd.read_csv('disaster_tweets/train.csv')
df_train.count()

In [ ]:
df_train.head()

In [ ]:
print(df_train['text'][177])
print(df_train['target'][177])

In [ ]:
import re
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))

def clean_text(text_string):
    text_string = text_string.lower()
    text_string = re.sub(r'(http\S+|[^\s\w])+', ' ', str(text_string))
    text_string = ' '.join([i for i in text_string.split() if i not in stop])
    return text_string

In [ ]:
df_train['text'] = df_train['text'].apply(clean_text)

In [ ]:
print(df_train['text'][177])
print(df_train['target'][177])

In [ ]:
print(df_train['text'][96])
print(df_train['target'][96])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# cuts off sms text after 100 words
maxlen = 100
# considers only the top 10000 words in the dataset
max_words = 10000

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df_train['text'].tolist())

sequences_train = tokenizer.texts_to_sequences(df_train['text'])

data_train = pad_sequences(sequences_train, maxlen=maxlen)
data_train[0]

In [ ]:
tokenizer.index_word

In [ ]:
labels_train = np.asarray(df_train['target'].to_list())
labels_train

In [ ]:
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.')
print('Shape of data tensor:', data_train.shape)
print('Shape of label tensor:', labels_train.shape)

![title](ann.png)

In [ ]:
from tensorflow.keras import models, layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(maxlen,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(data_train, labels_train, stratify=labels_train, test_size=0.10)

In [ ]:
history = model.fit(
    x_train,
    y_train,
    epochs=10,
    batch_size=128,
    validation_data=(x_val, y_val),
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            'disaster_tweets.h5', monitor='val_loss', verbose=1, save_best_only=True
        )
    ]
)

![title](brnn.png)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional

model = models.Sequential()
model.add(layers.Embedding(max_words, 32))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history = model.fit(
    x_train,
    y_train,
    epochs=10,
    batch_size=128,
    validation_data=(x_val, y_val),
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            'disaster_tweets.h5', monitor='val_loss', verbose=1, save_best_only=True
        )
    ]
)

In [ ]:
import matplotlib.pyplot as plt

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()

history_dict = history.history
acc_values = history_dict['accuracy']
acc_loss_values = history_dict['val_accuracy']

plt.plot(epochs, acc_values, 'bo', label='Training acc')
plt.plot(epochs, acc_loss_values, 'b', label='Validation acc')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
results = model.evaluate(x_val, y_val, verbose=2)

In [ ]:
from tensorflow import keras

load_model = keras.models.load_model('disaster_tweets.h5')

In [ ]:
def predict_disaster(disaster_text):
    disaster_text = clean_text(disaster_text)
    sequences = np.array(tokenizer.texts_to_sequences([disaster_text]))
    data = pad_sequences(sequences, maxlen=maxlen)
    predict = load_model.predict(data)
    return predict[0][0]

In [ ]:
predict_disaster("Australia is being ravaged by the worst wildfires seen in decades, with large swaths of the country devastated since the fire season began in late July.")

In [ ]:
predict_disaster("Heartbreaking! Medical workers fighting the #coronavirus on the front line in Wuhan fell asleep without taking off their protective suits. Many of them have been working nonstop and could only sleep for 2-3 hours a day.")

In [ ]:
predict_disaster("I love my mother")

In [ ]:
predict_disaster("Today we learned about Keras.")

![title](robot.jpg)